<a href="https://colab.research.google.com/github/omermehboob/NIDS-using-machine-learning-and-deep-learning/blob/main/DNN_4(NIDS_BINARY_7_features).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Refined_data_set(label_encoding_7_features).csv')

In [3]:
df.shape

(1336498, 9)

In [4]:
df.head()

,Unnamed: 0,Bwd Packet Length Std,Flow Bytes/s,Total Length of Fwd Packets,Fwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Total,Label
0,0,0.00000,2046,140.0,0.000000,7.124800e+04,3.0,29304.0,1
1,1,0.00000,7,30.0,0.000000,2.005113e+06,10.0,4016867.0,0
2,2,0.00000,2,12.0,3.464102,3.057131e+06,97.0,5318063.0,1
3,3,0.00000,1081081,64.0,0.000000,4.901360e+01,1.0,48.0,1
4,4,426.14309,57,2691.0,181.667939,4.191832e+06,39.0,117000000.0,1


In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
df.head()

,Bwd Packet Length Std,Flow Bytes/s,Total Length of Fwd Packets,Fwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Total,Label
0,0.00000,2046,140.0,0.000000,7.124800e+04,3.0,29304.0,1
1,0.00000,7,30.0,0.000000,2.005113e+06,10.0,4016867.0,0
2,0.00000,2,12.0,3.464102,3.057131e+06,97.0,5318063.0,1
3,0.00000,1081081,64.0,0.000000,4.901360e+01,1.0,48.0,1
4,426.14309,57,2691.0,181.667939,4.191832e+06,39.0,117000000.0,1


In [7]:
X=df[["Bwd Packet Length Std", "Flow Bytes/s", "Total Length of Fwd Packets", "Fwd Packet Length Std",
     "Flow IAT Std", "Flow IAT Min", "Fwd IAT Total"]]
y=df[["Label"]]

In [8]:
print(X.shape)
print(y.shape)

(1336498, 7)
(1336498, 1)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [10]:
scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)

scaler = Normalizer().fit(y_train)
y_train = scaler.transform(y_train)

In [11]:
model = Sequential()
model.add(Dense(600,input_dim=7,activation='relu'))  
model.add(Dropout(0.10))
model.add(Dense(400,activation='relu'))  
model.add(Dropout(0.10))
model.add(Dense(200,activation='relu'))  
model.add(Dropout(0.10))
model.add(Dense(100,activation='relu'))  
model.add(Dropout(0.10))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 600)               4800      
                                                                 
 dropout (Dropout)           (None, 600)               0         
                                                                 
 dense_1 (Dense)             (None, 400)               240400    
                                                                 
 dropout_1 (Dropout)         (None, 400)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               80200     
                                                                 
 dropout_2 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 100)               2

In [12]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=512, epochs=100)

Epoch 1/100
2089/2089 [==============================] - 15s 5ms/step - loss: 0.3442 - accuracy: 0.8556 - val_loss: 4741628.0000 - val_accuracy: 0.8257
Epoch 2/100
2089/2089 [==============================] - 9s 4ms/step - loss: 0.3287 - accuracy: 0.8604 - val_loss: 2503333.2500 - val_accuracy: 0.8262
Epoch 3/100
2089/2089 [==============================] - 9s 4ms/step - loss: 0.3219 - accuracy: 0.8629 - val_loss: 1035083.6250 - val_accuracy: 0.8521
Epoch 4/100
2089/2089 [==============================] - 9s 4ms/step - loss: 0.2854 - accuracy: 0.8731 - val_loss: 1092720.0000 - val_accuracy: 0.8244
Epoch 5/100
2089/2089 [==============================] - 9s 4ms/step - loss: 0.2594 - accuracy: 0.8856 - val_loss: 1528763.8750 - val_accuracy: 0.8263
Epoch 6/100
2089/2089 [==============================] - 9s 4ms/step - loss: 0.2501 - accuracy: 0.8925 - val_loss: 1261336.3750 - val_accuracy: 0.8254
Epoch 7/100
2089/2089 [==============================] - 9s 4ms/step - loss: 0.2406 - accurac

In [13]:
from sklearn import metrics
def decode_sentiment(score):
    return 1 if score>0.5 else 0

In [14]:
scores = model.predict(X_test)
y_pred = [decode_sentiment(score) for score in scores]
accuracy=metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy of DNN_4 is {accuracy:.2f}%')

Accuracy of DNN_4 is 0.82%


In [15]:
print("classification report:")
print(metrics.classification_report(y_test, y_pred))

classification report:
              precision    recall  f1-score   support

           0       0.64      0.00      0.00     47065
           1       0.82      1.00      0.90    220235

    accuracy                           0.82    267300
   macro avg       0.73      0.50      0.45    267300
weighted avg       0.79      0.82      0.75    267300

